In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense,Input
from keras import applications
from keras.models import Model

# dimensions of our images.
# img_width, img_height = 224, 224
img_width, img_height = 500, 500

pretrained_model = "inceptionv3_500"
weight_folder = "part_3_weights"


# top_model_weights_path = '../model/bottleneck_fc_model_%s.h5'%pretrained_model
# train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
# validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
# test_data_dir = '../data/Pandora18K_small_train_val_test_split/test'
# nb_train_samples = 1462
# nb_validation_samples = 167
# nb_test_samples = 171
train_data_dir = '../data/Pandora18K_train_val_test_split/train'
validation_data_dir = '../data/Pandora18K_train_val_test_split/val'
test_data_dir = '../data/Pandora18K_train_val_test_split/test'
nb_train_samples = 14313
nb_validation_samples = 1772
nb_test_samples = 1791

batch_size = 64


Using TensorFlow backend.


In [2]:
from keras.utils.np_utils import to_categorical
tag=''
train_data = np.load(open('../data/image_train%s.npy'%tag))
#full
train_labels = np.array([0]*684+[1]*598+[2]*655+[3]*657+[4]*808+[5]*675+[6]*715+[7]*946+[8]*995+[9]*1021+[10]*803+[11]*816+[12]*566+[13]*959+[14]*842+[15]*831+[16]*849+[17]*893)

#small
#train_labels = np.array([0]*78+[1]*79+[2]*77+[3]*85+[4]*78+[5]*87+[6]*80+[7]*81+[8]*81+[9]*80+[10]*85+[11]*83+[12]*74+[13]*87+[14]*83+[15]*83+[16]*78+[17]*83)

train_labels = to_categorical(train_labels, num_classes=18)

validation_data = np.load(open('../data/image_validation%s.npy'%tag))
#full
validation_labels = np.array([0]*72+[1]*73+[2]*72+[3]*93+[4]*78+[5]*74+[6]*85+[7]*124+[8]*131+[9]*118+[10]*109+[11]*105+[12]*80+[13]*130+[14]*108+[15]*89+[16]*111+[17]*120)

#small
#validation_labels = np.array([0]*8+[1]*10+[2]*11+[3]*5+[4]*11+[5]*6+[6]*8+[7]*8+[8]*9+[9]*12+[10]*7+[11]*10+[12]*14+[13]*5+[14]*11+[15]*11+[16]*12+[17]*9)
validation_labels = to_categorical(validation_labels, num_classes=18)

test_data = np.load(open('../data/image_test%s.npy'%tag))
#full
test_labels = np.array([0]*91+[1]*60+[2]*75+[3]*82+[4]*104+[5]*83+[6]*95+[7]*121+[8]*131+[9]*123+[10]*103+[11]*117+[12]*65+[13]*123+[14]*121+[15]*112+[16]*89+[17]*96)

#small
#test_labels = np.array([0]*14+[1]*11+[2]*12+[3]*10+[4]*11+[5]*7+[6]*12+[7]*11+[8]*10+[9]*8+[10]*8+[11]*7+[12]*12+[13]*8+[14]*6+[15]*6+[16]*10+[17]*8)
test_labels = to_categorical(test_labels, num_classes=18)

In [24]:
import keras
import os
import pickle




base_model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3))


In [28]:
pretrained_model = "inceptionv3_500"
weight_folder = "part_3_weights"

#0:lr,1:dense middle,2:dropout1, 3:dropout2
params=(1.951551576923019e-05, 650, 0.7037775003798115, 0.7698038979003217)
# params =(1.4813844087597416e-05, 593, 0.4553738549907012, 0.7257928399177316)
# params = (9.872134816612904e-05, 653, 0.40977067265920386, 0.7121187929915329)
# params = (1.3295116966818116e-06, 394, 0.4659839988579486, 0.751079719556778)

print(params)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dropout(params[2]))#0.6
top_model.add(Dense(params[1], activation='relu'))#256
top_model.add(Dropout(params[2]))#0.6
top_model.add(Dense(18, activation='softmax'))

weight_dir = "../model/%s"%weight_folder
best_weight_path = os.path.join(weight_dir, "best_weights_%s.h5"%pretrained_model)

print(best_weight_path)

top_model.load_weights(best_weight_path)


# for layers in top_model.layers:
#     trainable=False
    
# for layers in base_model.layers:
#     trainable=True

# top_model.compile(optimizer=keras.optimizers.SGD(lr=1e-5, momentum=0.9),
#                   loss='categorical_crossentropy', metrics=['accuracy'])

model = Model(inputs= base_model.input, outputs= top_model(base_model.output))
model.load_weights("../model/finetune/finetune_weights_0.5857.h5")




(1.951551576923019e-05, 650, 0.7037775003798115, 0.7698038979003217)
../model/part_3_weights/best_weights_inceptionv3_500.h5


In [5]:
# model.compile(optimizer=keras.optimizers.SGD(lr=0, momentum=0),
#                   loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
for i,layer in enumerate(model.layers):
    print (i, layer.name,layer)

(0, 'input_1', <keras.engine.topology.InputLayer object at 0x7fb5e8134cd0>)
(1, 'conv2d_1', <keras.layers.convolutional.Conv2D object at 0x7fb58f700d50>)
(2, 'batch_normalization_1', <keras.layers.normalization.BatchNormalization object at 0x7fb58f6b2450>)
(3, 'activation_1', <keras.layers.core.Activation object at 0x7fb5e813c090>)
(4, 'conv2d_2', <keras.layers.convolutional.Conv2D object at 0x7fb5e813c050>)
(5, 'batch_normalization_2', <keras.layers.normalization.BatchNormalization object at 0x7fb58f66cd10>)
(6, 'activation_2', <keras.layers.core.Activation object at 0x7fa912c91290>)
(7, 'conv2d_3', <keras.layers.convolutional.Conv2D object at 0x7fa912c4c790>)
(8, 'batch_normalization_3', <keras.layers.normalization.BatchNormalization object at 0x7fa912c3edd0>)
(9, 'activation_3', <keras.layers.core.Activation object at 0x7fa912c102d0>)
(10, 'max_pooling2d_1', <keras.layers.pooling.MaxPooling2D object at 0x7fa912bdc410>)
(11, 'conv2d_4', <keras.layers.convolutional.Conv2D object at 0x

In [29]:
pretrained_model = "part_3_weights"
weight_dir = "../model/%s"%pretrained_model
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)

best_weight_path = "../model/best.h5"
best_acc = 0
best_lr = 0
best_momentum = 0
best_frozen = 0
num_parameter_sets = 1
    
# for i in range(num_parameter_sets):
    
#     lr = 10**np.random.uniform(low=-10, high=-5)
    
#     momentum = np.random.uniform(low = 0.7, high = 0.9)
#     num_frozen = np.random.randint(300, high=312) #312 total

lr = 10**-8
momentum = 0.9
num_frozen = 312
print("lr = " + str(lr))
print("momentum = " + str(momentum))
print("number of frozen layers = " + str(num_frozen))

# for layer in model.layers:
#     layer.trainable = True

for layer in model.layers[:num_frozen]:
    layer.trainable = False
for layer in model.layers[280:]:
    layer.trainable = True
for layer in model.layers[311].layers:
    layer.trainable = True

    
model.layers[311].load_weights("../model/part_3_weights/best_weights_inceptionv3_500.h5")  
model.layers[311].compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=lr,momentum=0.9),
              metrics=['accuracy'])
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=lr,momentum=0.9),
              metrics=['accuracy'])
    
    
    

lr = 1e-08
momentum = 0.9
number of frozen layers = 312


In [30]:
loss, acc = model.evaluate(validation_data, validation_labels, batch_size=32, verbose=1, sample_weight=None)

1772/1772 [==============================] - 84s    


In [31]:
print((loss,acc))

(1.5215515156631814, 0.58577878130747019)


In [19]:
# for layer in model.layers:
#     if layer.trainable:
#         print(layer)
#     print (layer.name)
#     print('______')


    

<tf.Variable 'momentum_3:0' shape=() dtype=float32_ref>

In [22]:
# for layer in model.layers[311].layers:
#     print layer



In [23]:
# model.fit_generator(
#     train_generator,
#     steps_per_epoch=(nb_train_samples // batch_size)+1,
#     epochs=3,
#     validation_data=validation_generator,
#     validation_steps=(nb_validation_samples // batch_size)+1, pickle_safe=True, workers=1)


In [13]:
epochs =10
weight_path = "../model/finetune/finetune_weights.h5"
checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path, verbose=1,monitor='val_acc', save_best_only=True)
model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels),callbacks=[checkpointer])

Train on 14313 samples, validate on 1772 samples
Epoch 1/10
 1856/14313 [==>...........................] - ETA: 634s - loss: 3.2256 - acc: 0.3955

KeyboardInterrupt: 

In [ ]:
loss, acc = model.evaluate(validation_data, validation_labels, batch_size=32, verbose=1, sample_weight=None)

In [ ]:
print((loss, acc))

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)
model = Sequential()
model.add(Dropout(0,input_shape = (img_width,img_height,3)))


generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
img_train = model.predict_generator(
    generator, nb_train_samples // 32+1, verbose=True, pickle_safe=True, workers=1)
del model
np.save(open('../data/image_train.npy', 'w'),
        img_train)